In [1]:
from arcgis import GIS
import pandas as pd
from arcgis.features import GeoAccessor

/opt/conda/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
gis = GIS(url = 'https://NSOE.maps.arcgis.com', username = 'anu4_790')

Enter password: ········


In [3]:
sdf_Exit = GeoAccessor.from_featureclass('../Data/MJBA/Exits.shp')
sdf_flood = GeoAccessor.from_featureclass('../Data/processed/exits_flood.shp')
sdf_EucDist = GeoAccessor.from_featureclass('../Data/processed/Exits_distance_to_DCFC.shp')
sdf_AmenityCount=GeoAccessor.from_featureclass('../Data/processed/exits_amenities.shp')
sdf_DriveDist = GeoAccessor.from_featureclass('../Data/processed/exits_distances.shp')
sdf_DistToPower = GeoAccessor.from_featureclass('../Data/processed/Exits_distance_to_Power.shp')

In [4]:
#Remove extraneous columns 
sdf_Exit = sdf_Exit[['FID', 'Longitude', 'Latitude', 'County', 'SHAPE']]
sdf_flood = sdf_flood[['FID', 'Flood_cat']]
sdf_EucDist = sdf_EucDist[['FID', 'Dist_to_DC']]
sdf_AmenityCount = sdf_AmenityCount[['FID', 'amenity_n']]
sdf_DriveDist = sdf_DriveDist[['FID', 'distance']]
sdf_DistToPower = sdf_DistToPower[['FID', 'Dist_to_Pw']]

In [5]:
#Set index to FID
sdf_Exit.set_index(['FID'],inplace=True)
sdf_flood.set_index(['FID'],inplace=True)
sdf_EucDist.set_index(['FID'],inplace=True)
sdf_AmenityCount.set_index(['FID'],inplace=True)
sdf_DriveDist.set_index(['FID'],inplace=True)
sdf_DistToPower.set_index(['FID'],inplace=True)

In [6]:
#Join Data to a master geodataframe
sdf_Master = pd.merge(left=sdf_Exit,right=sdf_flood,how='left',left_index=True, right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_EucDist,how='left',left_index=True, right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_AmenityCount,how='left',left_index=True, right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_DriveDist,how='left',left_index=True, right_index=True)
sdf_Master = pd.merge(left=sdf_Master,right=sdf_DistToPower,how='left',left_index=True, right_index=True)


In [7]:
#Fix Euc Dist - change from meters to miles
sdf_Master['Dist_to_DC'] = sdf_Master['Dist_to_DC']/1609.34

In [8]:
sdf_Master

,Longitude,Latitude,County,SHAPE,Flood_cat,Dist_to_DC,amenity_n,distance,Dist_to_Pw
FID,,,,,,,,,
7059,-84.314304,35.019492,Cherokee,"{'x': -84.31430368976345, 'y': 35.019491849669...",Area of Minimal Flood Hazard,103.022794,0,NaN,5.987276
7060,-84.311492,35.014933,Cherokee,"{'x': -84.31149165991212, 'y': 35.014933379571...",Area of Minimal Flood Hazard,102.983699,0,NaN,5.879956
7061,-84.295528,35.013561,Cherokee,"{'x': -84.29552778977563, 'y': 35.013560589649...",Area of Minimal Flood Hazard,102.169982,0,NaN,5.014686
7062,-84.280291,35.012880,Cherokee,"{'x': -84.28029090010494, 'y': 35.012879950349...",Area of Minimal Flood Hazard,101.379188,0,NaN,4.193707
7063,-84.275190,35.017816,Cherokee,"{'x': -84.27519011992803, 'y': 35.017816120417...",Area of Minimal Flood Hazard,100.986949,0,NaN,3.823759
...,...,...,...,...,...,...,...,...,...
8232,-78.518206,35.389884,Johnston,"{'x': -78.51820576978042, 'y': 35.389883880137...",Area of Minimal Flood Hazard,14.067219,0,30.408900,1.610386
8233,-78.289842,35.519484,Johnston,"{'x': -78.28984227969744, 'y': 35.519484229613...",Area of Minimal Flood Hazard,1.623417,6,1.785973,0.400149
8234,-80.181938,35.809700,Davidson,"{'x': -80.18193820998039, 'y': 35.809699620441...",Area of Minimal Flood Hazard,20.637721,1,27.472599,2.379980


In [9]:
sdf_Master.spatial.to_featureclass('../Data/processed/Final_exits.shp')

'../Data/processed/Final_exits.shp'

In [10]:
#Import NREL CSV as dataframe
DCFC_df = pd.read_csv('../Data/NREL/DCFC.csv')

In [11]:
#Convert dataframe to arcgis spatial dataframe
DCFC_sdf = GeoAccessor.from_xy(DCFC_df, x_column='longitude',y_column='latitude',sr=4326)

In [12]:
#Convert to feature layer
DCFC_lyr = DCFC_sdf.spatial.to_featurelayer(title = "NREL DC Fast Charger Site",
                                   tags="ENV790, EV Charger, NREL")

In [13]:
DCFC_lyr

<Item title:"NREL DC Fast Charger Site" type:Feature Layer Collection owner:anu4_790>

In [14]:
#Construct dictionary of feature layer properties
featLayer_properties = {
    'snippet':"NREL DC Fast Charging site. Extracted and subset via the NREL API",
    'summary': "These data were extracted from the NREL API, subset using the following criteria"
}

#Apply the changes
DCFC_lyr.update(featLayer_properties)

True

In [15]:
DCFC_lyr.share(org=True)

{'results': [{'itemId': '099769bfecb245529ed5f79fb987ed95',
   'success': True,
   'notSharedWith': []}]}

In [16]:
sdf_Master.spatial.to_featurelayer(title="Candidate Charger Sites",
                                  tags = "ENV790, EV Charger, NREL")

<Item title:"Candidate Charger Sites" type:Feature Layer Collection owner:anu4_790>

In [17]:
#Import Amenities as dataframe
sdf_Amenities = GeoAccessor.from_featureclass('../Data/OSM/OSM_amenities.shp')


In [18]:
sdf_Amenities.spatial.to_featurelayer(title="OSM Amenities",
                                  tags = "ENV790, EV Charger, NREL")

<Item title:"OSM Amenities" type:Feature Layer Collection owner:anu4_790>

In [22]:
sdf_Substations = GeoAccessor.from_featureclass('../Data/HIFLD/substations.shp')

In [23]:
sdf_Substations=sdf_Substations[['index','ID','SHAPE', 'LATITUDE','LONGITUDE']]

In [24]:
#Not sure why this won't work
#sdf_Substations.spatial.to_featurelayer(title="Substations",tags = "ENV790, EV Charger, NREL")


Exception: Job failed.